# Social statistics

In [ ]:
import pandas as pd
X_train_path = "data/X_train_Hi5.csv"
X_train = pd.read_csv(X_train_path)


In [ ]:
# Extract the water withdrawal for 2021
X_train["date"] = pd.to_datetime(X_train["piezo_measurement_date"])
X_2021 = X_train[X_train["date"].dt.year == 2021]

X_2021["total_withdraw"] = (X_2021["prelev_other_volume_sum"] + X_2021["prelev_volume_0"] + X_2021["prelev_volume_1"] + X_2021["prelev_volume_2"]) / 123396566321 *300
prelev_df = []
for prelev in range(3):
    selected_cols = [f"prelev_longitude_{prelev}", f"prelev_latitude_{prelev}",f"prelev_volume_{prelev}", f"prelev_usage_label_{prelev}"]
    sub_prelev_df = X_2021[selected_cols]
    sub_prelev_df.columns = ["prelev_longitude","prelev_latitude","prelev_volume", "prelev_usage_label"]
    prelev_df.append(sub_prelev_df)

prelev_df = pd.concat(prelev_df, axis=0)

prelev_df.shape

In [ ]:
prelev_df = prelev_df.drop_duplicates()
print(prelev_df.shape)
print(X_2021["total_withdraw"].max())


In [ ]:
# Prepare for plotting
# Size
prelev_df["point_size"] = 300* prelev_df["prelev_volume"] / prelev_df["prelev_volume"].max()
# categroy
prelev_cat = list(prelev_df['prelev_usage_label'].unique())
print(prelev_cat)
prelev_cat_dic = {name : index for index, name in enumerate(prelev_cat)}
prelev_df["cat"] = prelev_df["prelev_usage_label"].apply(lambda x: prelev_cat_dic.get(x, 6))
prelev_df

In [ ]:
prelev_cat_dic

In [ ]:
# cartopy to display maps
%pip install cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

In [ ]:
#ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
def display_France_map(ax):
    """Display the map of France"""
    
    ax.set_extent([-5, 10, 42, 52], crs=ccrs.PlateCarree())
    # Draw the background
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.add_feature(cfeature.RIVERS)
    ax.gridlines(draw_labels=True)

In [ ]:
fig = plt.figure(figsize=(20, 20),  layout="constrained")
#
ax_1 = fig.add_subplot(121,  projection = ccrs.PlateCarree())
display_France_map(ax_1)
ax_1.set_title("Stations")
ax_1.scatter(data= X_2021, x="piezo_station_longitude", y="piezo_station_latitude", s="total_withdraw")

# Right prelev
ax_2 = fig.add_subplot(122,  projection = ccrs.PlateCarree())
display_France_map(ax_2)
ax_2.set_title("Main withdrawals of 2021")
scatter = ax_2.scatter(data=prelev_df, x="prelev_longitude", y="prelev_latitude", s="point_size", c='cat')
legend2 = ax_2.legend(*scatter.legend_elements(), loc="best", title="Usage")
ax_2.add_artist(legend2)
ax_2.legend
plt.show()